# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
from scipy.stats import linregress

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
weather_file = "weather.csv"
weather_csv = pd.read_csv(weather_file)
weather_csv.head()


,city,temp,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Burley,0.37,42.5357,-113.7928,0.98,66,20,12.35,US,1649795105
1,Saint Paul Harbor,5.73,57.7900,-152.4072,5.73,75,20,4.12,US,1649794582
2,Clyde River,-15.87,70.4692,-68.5914,-15.87,78,100,8.23,CA,1649794727
3,Faanui,27.96,-16.4833,-151.7500,27.96,72,7,7.04,PF,1649794668
4,Sitka,5.94,57.0531,-135.3300,5.94,28,20,3.60,US,1649794830


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key = g_key)

In [14]:
# Configure gmaps

weather_csv = weather_csv.dropna()

# Bring in location data pairs, humidity data, define map and layer, and add layer to map

locations = weather_csv[['lat', 'lng']]
Humidity = weather_csv['Humidity']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
len(locations)

587

In [16]:
len(Humidity)

587

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
perf_weather = weather_csv

#A max temperature lower than 80 degrees but higher than 70. (26 in cel = 80 F, and 21 in cel = 70 F)
perf_weather = perf_weather.loc[(perf_weather["Max Temp"] < 26) & (perf_weather["Max Temp"] > 21) ]   

#Wind speed less than 10 mph.
perf_weather = perf_weather.loc[perf_weather["Wind Speed"] < 10]   

#Zero cloudiness.
perf_weather = perf_weather.loc[perf_weather["Cloudiness"] == 0] 

#Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.
perf_weather = perf_weather.dropna()
perf_weather

,city,temp,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
144,Qena,22.39,26.1615,32.7183,22.39,21,0,5.66,EG,1649795141
204,Sakakah,21.76,29.9697,40.2064,21.76,22,0,4.32,SA,1649795157
238,Mossamedes,24.02,-15.1961,12.1522,24.02,75,0,1.96,AO,1649794715
253,Bethel,20.37,41.3712,-73.4140,21.46,52,0,5.14,US,1649794661
321,Marzuq,23.46,14.4000,46.4667,23.46,23,0,3.15,YE,1649794743
432,Bārmer,24.29,25.7500,71.3833,24.29,44,0,3.80,IN,1649795226
433,Luangwa,22.65,-15.6167,30.4167,22.65,74,0,0.42,ZM,1649795227
443,Gladstone,22.96,-23.8500,151.2500,22.96,78,0,2.57,AU,1649795230
588,Lao Cai,22.37,22.4833,103.9500,22.37,82,0,0.33,VN,1649795289


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Create dataframe and empty column



In [20]:
# Loop through dataframe and add nearby hotel result

hotel_df = perf_weather

params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #find lat and lng
    lat= row["lat"]
    lng = row["lng"]

    # add keyword
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # assemble url and make API request
    print(f"Retrieving Results {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("------------")

Retrieving Results 144: Qena.
Missing field/result... skipping.
------------
Retrieving Results 204: Sakakah.
Missing field/result... skipping.
------------
Retrieving Results 238: Mossamedes.
Closest hotel is Hotel Chik Chik Namibe.
------------
Retrieving Results 253: Bethel.
Missing field/result... skipping.
------------
Retrieving Results 321: Marzuq.
Missing field/result... skipping.
------------
Retrieving Results 432: Bārmer.
Missing field/result... skipping.
------------
Retrieving Results 433: Luangwa.
Missing field/result... skipping.
------------
Retrieving Results 443: Gladstone.
Missing field/result... skipping.
------------
Retrieving Results 588: Lao Cai.
Closest hotel is Legend Hotel.
------------


In [21]:
hotel_df

,city,temp,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
144,Qena,22.39,26.1615,32.7183,22.39,21,0,5.66,EG,1649795141,"Grand hotel ""Quena"""
204,Sakakah,21.76,29.9697,40.2064,21.76,22,0,4.32,SA,1649795157,NaN
238,Mossamedes,24.02,-15.1961,12.1522,24.02,75,0,1.96,AO,1649794715,Hotel Chik Chik Namibe
253,Bethel,20.37,41.3712,-73.4140,21.46,52,0,5.14,US,1649794661,NaN
321,Marzuq,23.46,14.4000,46.4667,23.46,23,0,3.15,YE,1649794743,NaN
432,Bārmer,24.29,25.7500,71.3833,24.29,44,0,3.80,IN,1649795226,NaN
433,Luangwa,22.65,-15.6167,30.4167,22.65,74,0,0.42,ZM,1649795227,NaN
443,Gladstone,22.96,-23.8500,151.2500,22.96,78,0,2.57,AU,1649795230,NaN
588,Lao Cai,22.37,22.4833,103.9500,22.37,82,0,0.33,VN,1649795289,Legend Hotel


In [23]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure



Figure(layout=FigureLayout(height='420px'))